In [73]:
import os
import config
import zipfile
import shutil
import py_hydroweb
from glob import glob
import geopandas as gpd


In [36]:
dir_down = "data/altimetry-water-level/hydroweb"
file_name = 'hydroweb_lakes.zip'
region = [66, 26, 106, 45]    ### left, bottom, right, top


In [ ]:
### download data
client = py_hydroweb.Client("https://hydroweb.next.theia-land.fr/api", config.hydroweb_key)  
basket = py_hydroweb.DownloadBasket("my_download_basket")
basket.add_collection("HYDROWEB_LAKES_OPE", bbox=region)  ## left, bottom, right, top
client.submit_and_download_zip(basket, zip_filename=file_name, output_folder = dir_down)


Wait: 100%|██████████| 100/100 [00:02<00:00, 35.59it/s]
hydroweb_lakes.zip: 100%|██████████| 361k/361k [00:02<00:00, 156kB/s]


'data/altimetry-water-level/hydroweb/hydroweb_lakes.zip'

In [80]:
### unzip the download data
dest_dir = dir_down+'/hydroweb_lakes'
source_dir = dest_dir+'/HYDROWEB_LAKES_OPE/HYDROWEB_LAKES_OPE'
if not os.path.exists(dest_dir): 
    os.makedirs(dest_dir, exist_ok=True)
with zipfile.ZipFile(dir_down+'/'+file_name, 'r') as zip_ref:
    zip_ref.extractall(dest_dir)
for txt_name in os.listdir(source_dir):
    if txt_name.endswith('.txt'):
        shutil.move(os.path.join(source_dir, txt_name), os.path.join(dest_dir, txt_name))
shutil.rmtree(dest_dir+'/HYDROWEB_LAKES_OPE')


#### Preprocessing
Write the dahiti information into geopandas dataframe.

In [83]:
from shapely.geometry import Point
lake_id, lake_name, lake_contry, lake_basin, lake_lat, lake_lon = [], [], [], [], [], []
for path_file in paths_file:
    with open(path_file, 'r') as f:
        content = f.read()
        # Extract lake information
        lake_info = {}
        line_first = content.split('\n')[0]
        parts = line_first.split(';')
        lake_id.append(parts[10].split('=')[1].strip())
        lake_name.append(parts[0].split('=')[1])
        lake_contry.append(parts[1].split('=')[1])
        lake_basin.append(parts[2].split('=')[1])
        lake_lat.append(float(parts[3].split('=')[1]))  
        lake_lon.append(float(parts[4].split('=')[1]))


lake_info = {'id': lake_id, 'name': lake_name, 'contry': lake_contry, 'basin': lake_basin}
geo = [Point(lon, lat) for lon, lat in zip(lake_lon, lake_lat) ]

In [ ]:
lake_hydroweb_gdf = gpd.GeoDataFrame(lake_info, geometry=geo)
lake_hydroweb_gdf.head(2)
# lake_hydroweb_gdf.to_file(dir_down+'/hydroweb_lakes/hydroweb_hma.gpkg', driver='GPKG') 
